In [29]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# carga fichero

In [30]:
try:
    df_trabajo_fusion = pd.read_csv("../output/Datos_Fusionadosv1.csv")

except Exception as e:
    print(f"Error al cargar los datos: {str(e)}")

In [31]:
df_trabajo_fusion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9951 entries, 0 to 9950
Data columns (total 20 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   ID_Orden                                9951 non-null   int64  
 1   ID_Equipo                               9951 non-null   int64  
 2   Fecha                                   9951 non-null   object 
 3   Tipo_Mantenimiento                      9951 non-null   object 
 4   Costo_Mantenimiento                     9951 non-null   float64
 5   Duracion_Horas                          9951 non-null   int64  
 6   Ubicacion                               9951 non-null   object 
 7   Frecuencia de mantenimiento correctivo  9951 non-null   int64  
 8   ID_Registro                             9951 non-null   int64  
 9   Temperatura_C                           9951 non-null   float64
 10  Vibracion_mm_s                          9951 non-null   floa

# Convertir Fecha a año mes dia

In [32]:

df_trabajo_fusion['Fecha'] = pd.to_datetime(df_trabajo_fusion['Fecha'])

# Extraer año, mes y día de la columna 'Fecha'
df_trabajo_fusion['Año_O'] = df_trabajo_fusion['Fecha'].dt.year
df_trabajo_fusion['Mes_O'] = df_trabajo_fusion['Fecha'].dt.month
df_trabajo_fusion['Día_O'] = df_trabajo_fusion['Fecha'].dt.day

In [33]:

# Convertir las columnas 'Fecha' y 'Fecha_Revision' a formato datetime
df_trabajo_fusion['Fecha'] = pd.to_datetime(df_trabajo_fusion['Fecha'])
df_trabajo_fusion['Fecha_Revision'] = pd.to_datetime(df_trabajo_fusion['Fecha_Revision'])

# Calcular la diferencia en horas entre 'Fecha' y 'Fecha_Revision'
df_trabajo_fusion['Horas_Desde_Preventivo'] = (df_trabajo_fusion['Fecha_Revision'] - df_trabajo_fusion['Fecha']).dt.total_seconds() / 3600

print(df_trabajo_fusion.head())

   ID_Orden  ID_Equipo               Fecha Tipo_Mantenimiento  \
0         1         36 2020-01-01 00:00:00         Preventivo   
1         2        244 2020-01-01 01:00:00         Preventivo   
2         3        297 2020-01-01 02:00:00         Preventivo   
3         4        431 2020-01-01 03:00:00         Correctivo   
4         5        403 2020-01-01 04:00:00         Correctivo   

   Costo_Mantenimiento  Duracion_Horas     Ubicacion  \
0              6754.12               3  Planta Norte   
1              4274.63              35  Planta Norte   
2              5752.00              25  Planta Norte   
3              5690.14               8    Planta Sur   
4              7048.79              28  Planta Norte   

   Frecuencia de mantenimiento correctivo  ID_Registro  Temperatura_C  ...  \
0                                       9          307         139.97  ...   
1                                      11         1156         141.25  ...   
2                                     

### Eliminar columnas irrelevantes

In [41]:
# Eliminar columnas irrelevantes (iniciales hay que añadir mas)

discarded_columns = ["Modelo", "Fabricante", "ID_Registro", "Ubicacion", "Fecha", "Fecha_Revision", "ID_Orden"]
df_trabajo_fusion_limpio = df_trabajo_fusion.drop(discarded_columns, axis=1)
df_trabajo_fusion_limpio.head()

,ID_Equipo,Tipo_Mantenimiento,Costo_Mantenimiento,Duracion_Horas,Frecuencia de mantenimiento correctivo,Temperatura_C,Vibracion_mm_s,Horas_Operativas,Mes,Horas_Operativas_mean,Tipo_Equipo,Potencia_kW,Horas_Recomendadas_Revision,Año_O,Mes_O,Día_O,Horas_Desde_Preventivo
0,36,Preventivo,6754.12,3,9,139.97,4.31,41114.0,2020-01,41114.0,Motor,4351.0,3549,2020,1,1,0.0
1,244,Preventivo,4274.63,35,11,141.25,4.82,93042.0,2020-02,93042.0,Generador,1206.0,1061,2020,1,1,1061.0
2,297,Preventivo,5752.00,25,9,71.11,3.24,42929.0,2020-01,53789.6,Transformador,3728.0,8016,2020,1,1,8016.0
3,431,Correctivo,5690.14,8,14,89.23,4.12,81714.0,2020-01,81714.0,Transformador,253.0,7764,2020,1,1,8015.0
4,403,Correctivo,7048.79,28,8,61.95,4.66,96881.0,2020-01,72512.0,Motor,2597.0,4993,2020,1,1,8014.0


# Conversion datos categoricos

In [42]:
# Extraer columnas numéricas y categóricas
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(df_trabajo_fusion_limpio)
categorical_columns = categorical_columns_selector(df_trabajo_fusion_limpio)

print("Numerical columns", numerical_columns)
print("Categorical columns", categorical_columns)

Numerical columns ['ID_Equipo', 'Costo_Mantenimiento', 'Duracion_Horas', 'Frecuencia de mantenimiento correctivo', 'Temperatura_C', 'Vibracion_mm_s', 'Horas_Operativas', 'Horas_Operativas_mean', 'Potencia_kW', 'Horas_Recomendadas_Revision', 'Año_O', 'Mes_O', 'Día_O', 'Horas_Desde_Preventivo']
Categorical columns ['Tipo_Mantenimiento', 'Mes', 'Tipo_Equipo']


# Conversion de datos categoricos

In [43]:
# Crear el LabelEncoder
label_encoders = {}

# Ajustar y transformar las columnas categóricas
for column in categorical_columns:
    le = LabelEncoder()
    df_trabajo_fusion_limpio[column] = le.fit_transform(df_trabajo_fusion_limpio[column])
    label_encoders[column] = le

print(df_trabajo_fusion_limpio.head())

   ID_Equipo  Tipo_Mantenimiento  Costo_Mantenimiento  Duracion_Horas  \
0         36                   1              6754.12               3   
1        244                   1              4274.63              35   
2        297                   1              5752.00              25   
3        431                   0              5690.14               8   
4        403                   0              7048.79              28   

   Frecuencia de mantenimiento correctivo  Temperatura_C  Vibracion_mm_s  \
0                                       9         139.97            4.31   
1                                      11         141.25            4.82   
2                                       9          71.11            3.24   
3                                      14          89.23            4.12   
4                                       8          61.95            4.66   

   Horas_Operativas  Mes  Horas_Operativas_mean  Tipo_Equipo  Potencia_kW  \
0           41114.0    0   

# Guardado Intermedio

In [44]:
# Guardar el dataframe final mezclado en un nuevo archivo CSV
df_trabajo_fusion_limpio.to_csv('../output/Datos_Fusionadosv2.csv', index=False)

# Datos escalados

In [45]:
# Seleccionar las características y la variable objetivo
X = df_trabajo_fusion_limpio.drop(columns=['Tipo_Mantenimiento'])
y = df_trabajo_fusion_limpio['Tipo_Mantenimiento']

# Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convertir el array escalado a DataFrame
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Añadir la variable objetivo al DataFrame escalado
X_scaled_df['Tipo_Mantenimiento'] = y.values

# Mostrar las primeras filas del DataFrame escalado
print(X_scaled_df.head())


   ID_Equipo  Costo_Mantenimiento  Duracion_Horas  \
0  -1.481601             0.574740       -1.541048   
1  -0.035868            -0.278163        0.813797   
2   0.332516             0.230027        0.077908   
3   1.263902             0.208749       -1.173104   
4   1.069284             0.676101        0.298674   

   Frecuencia de mantenimiento correctivo  Temperatura_C  Vibracion_mm_s  \
0                               -0.463810       1.360251        1.190803   
1                                0.211446       1.403471        1.544833   
2                               -0.463810      -0.964825        0.448034   
3                                1.224330      -0.352999        1.058909   
4                               -0.801438      -1.274115        1.433765   

   Horas_Operativas       Mes  Horas_Operativas_mean  Tipo_Equipo  \
0         -0.290706 -1.667356              -0.368066     0.442590   
1          1.498749 -1.398662               1.818257    -0.443926   
2         -0.2281

# Guardado Final

In [46]:
# Guardar el dataframe final mezclado en un nuevo archivo CSV
df_trabajo_fusion_limpio.to_csv('../output/Datos_Fusionadosv2_escalado.csv', index=False)